<a href="https://colab.research.google.com/github/SPVillacorta/GeoNER-SchemaEval/blob/main/geoner_schema_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating the Performance of a Trained NER Model using a Confusion Matrix

This notebook demonstrates how to evaluate the performance of a pre-trained Named Entity Recognition (NER) model using a confusion matrix and classification report. It specifically uses the Flair NLP framework and `scikit-learn` for evaluation.

**Important Note:**
For demonstration and reproducibility, this notebook expects the following files to be accessible within a cloned repository structure (or downloaded/provided):
* A pre-trained model: `./ozrock/model/best-model.pt`
* Example PDF documents: `./ozrock/IRpdfs/`
* Annotated dataset: `./ozrock/annotated-ozrock.csv`

In [ ]:
## 0. Setup and Dependencies

First, we need to install the required libraries and download necessary NLTK data.

```python
# Install required libraries
!pip install flair pdfplumber pdfminer scikit-learn pandas seaborn matplotlib numpy

# Import necessary modules
import glob
import nltk
import matplotlib.pyplot as plt
import numpy as np
import os
import pdfplumber
import pdfminer
import pandas as pd
import seaborn as sns
import flair
from flair.data import Corpus, Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from pdfminer.pdfdocument import PDFNoValidXRef
from pdfminer.psparser import PSEOF
from sklearn.metrics import confusion_matrix, classification_report
from typing import List

# Download NLTK data
nltk.download("punkt")

1. Prepare Your Environment and Data (Google Colab Specific)
When running this notebook on Google Colab, you need to clone the GitHub repository to access the model, example PDFs, and annotated data.

In [ ]:
# Clone the GitHub repository
# Replace 'SPVillacorta/GeoNER-SchemaEval' with your actual GitHub username and repository name if different
!git clone [https://github.com/SPVillacorta/GeoNER-SchemaEval.git](https://github.com/SPVillacorta/GeoNER-SchemaEval.git)

# Change the current working directory to the cloned repository's root.
# This ensures all relative paths (e.g., './ozrock/...') work correctly.
import os
os.chdir("GeoNER-SchemaEval") # Make sure this matches your repository's folder name
print(f"Current working directory set to: {os.getcwd()}")
!ls -F # Verify the contents of the current directory; you should see 'ozrock/'

2. PDF to Sentences Conversion
This function extracts text from PDF files within a specified directory and tokenizes it into sentences.

In [ ]:
def pdf_to_sentences(pdf_dir: str):
    pdf_paths = glob.glob(os.path.join(pdf_dir, "*.pdf"))
    all_sentences = []
    for pdf_path in pdf_paths:
        try:
            with pdfplumber.open(pdf_path) as pdf:
                text = "\n".join([page.extract_text() for page in pdf.pages])
                sentences = nltk.sent_tokenize(text)
                all_sentences.extend(sentences)
        except (pdfminer.pdfdocument.PDFNoValidXRef, pdfminer.psparser.PSEOF) as e:
            print(f"Warning: Skipping file {pdf_path} due to an error. {e}")
    return all_sentences

3. Load the Pre-trained Model
Load the pre-trained Flair SequenceTagger model from the specified path.
Due to the model's size, it cannot be hosted directly on GitHub. It will be downloaded from an external service.

In [ ]:
# *** IMPORTANT: Replace 'YOUR_GOOGLE_DRIVE_FILE_ID_HERE' with your actual Google Drive file ID ***
# To get the ID:
# 1. Upload 'best-model.pt' to your Google Drive.
# 2. Right-click the file and select "Share" -> "Share".
# 3. Change access to "Anyone with the link".
# 4. Copy the link. The ID is the part between /d/ and /view.
#    Example: [https://drive.google.com/file/d/THIS_IS_THE_ID/view?usp=sharing](https://drive.google.com/file/d/THIS_IS_THE_ID/view?usp=sharing)
model_file_id = 'YOUR_GOOGLE_DRIVE_FILE_ID_HERE'

# Local path where the model will be saved within the Colab environment
model_local_path = './ozrock/model/best-model.pt'

# Ensure the destination folder exists
!mkdir -p ./ozrock/model

# Download the model
print(f"Downloading model from Google Drive (ID: {model_file_id})... Make sure the file is shared with 'Anyone with the link'.")
!gdown --id {model_file_id} -O {model_local_path}
print("Model downloaded successfully.")

# Load the model from the local path
model = SequenceTagger.load(model_local_path)

4. Get Sentences from Example PDF Files
Specify the directory containing your example PDF files and extract sentences using the defined function.

In [ ]:
PDF_DIR = "./ozrock/IRpdfs"
sentences = pdf_to_sentences(PDF_DIR)
print(f"Extracted {len(sentences)} sentences from example PDFs.")

5. Load the Annotated Dataset
Load your gold-standard annotated data from a CSV file. This data will be used as the true labels for evaluation.

In [ ]:
annotated_data = pd.read_csv("./ozrock/annotated-ozrock.csv")
labeled_data = []
for index, row in annotated_data.iterrows():
    text = row["Text"]
    labels = row["new_label"].split(",") # Assuming labels are comma-separated
    sentence = Sentence(text, use_tokenizer=True)
    tokens = [token.text for token in sentence.tokens]
    labeled_data.append((tokens, labels))
print(f"Loaded {len(labeled_data)} annotated instances.")

6. Predict Labels
Use the loaded model to predict NER tags for the sentences from your annotated dataset.

In [ ]:
predicted_sentences = []
for tokens, _ in labeled_data: # We only need tokens for prediction
    sentence = Sentence(" ".join(tokens))
    model.predict(sentence)
    predicted_sentences.append(sentence)

# Prepare true_tags by extracting only the labels from labeled_data
true_tags = [labels for _, labels in labeled_data]
print(f"Generated predictions for {len(predicted_sentences)} sentences.")

7. Evaluation Function
This function calculates and visualizes the confusion matrix and prints the classification report for the NER model's performance.

In [ ]:
def evaluate(predictions: List[Sentence], true_values: List[List[str]]) -> None:
    y_pred = []
    y_true = []
    skipped_instances = 0

    for i, (pred_sentence, true_label_list) in enumerate(zip(predictions, true_values)):
        pred_tags = [span.tag for span in pred_sentence.get_spans("ner")]

        # Ensure lengths match for a valid comparison
        if len(pred_tags) != len(true_label_list):
            skipped_instances += 1
            continue

        y_pred.extend(pred_tags)
        y_true.extend(true_label_list)

    print(f"Skipped {skipped_instances} instances due to mismatched lengths.")

    # Calculate the confusion matrix and classification report
    labels = sorted(list(set(y_true) | set(y_pred)))

    cm = confusion_matrix(y_true, y_pred, labels=labels, zero_division=0)
    report = classification_report(y_true, y_pred, labels=labels, zero_division=0)

    # Print the classification report
    print("\nClassification Report:")
    print(report)

    # Plot the confusion matrix using Seaborn
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix for NER Model Evaluation")
    plt.tight_layout()

    # Save the figure with high DPI
    plt.savefig('cm_ozrock_7pdfs.png', dpi=600)
    # plt.show() # Uncomment to display the plot inline if running locally

8. Run Evaluation
Execute the evaluation function with the predicted and true labels.

In [ ]:
evaluate(predicted_sentences, true_tags)